# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# load the csv file from WeatherPy and create dataframe
file = '../WeatherPy/output_data/cities.csv'

cities_data = pd.read_csv(file)

cities_df = pd.DataFrame(cities_data)

cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Gamba,-2.65,10.00,77.43,87,84,8.12,GA,1603090226
1,Nikolskoye,59.70,30.79,37.99,93,75,6.71,RU,1603090227
2,Samarai,-10.62,150.67,80.62,86,94,12.08,PG,1603090227
3,Fayaoué,-20.65,166.53,75.13,74,19,9.40,NC,1603090227
4,Ilulissat,69.22,-51.10,42.80,70,100,10.29,GL,1603090228


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# configure gmaps
gmaps.configure(api_key=g_key)

# create locations and humidity weight variable
marker_locations = cities_df[['Lat', 'Lng']]
humidity = cities_df['Humidity']


In [13]:
fig = gmaps.figure()

# add heat layer with marker locations and humidity as weight
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, dissipating=False,
                              max_intensity=cities_df['Humidity'].max().astype(float), point_radius=2.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
# Ideal weather condition:
# - Max Temperature: 70 - 80
# - Wind Speeds: <= 15
# - Cloudiness: < 50

droprows = []

# find index of rows that do not match the following conditions
for i in cities_df.index:
    # add index of cities with max temps below 70 or greater than 80
    if (cities_df['Max Temp'][i] < 70) | (cities_df['Max Temp'][i] > 80):
        droprows.append(i)
    # add index of cities with wind speeds greater than 15
    elif (cities_df['Wind Speed'][i] > 15):
        droprows.append(i)
    elif (cities_df['Cloudiness'][i] >= 50):
        droprows.append(i)

ideal_conditions_df = cities_df.drop(droprows)

# drop all rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

ideal_conditions_df.head()
    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Fayaoué,-20.65,166.53,75.13,74,19,9.40,NC,1603090227
9,Lakkí,37.13,26.85,71.60,78,20,5.82,GR,1603090230
15,Mahébourg,-20.41,57.70,78.80,65,40,14.99,MU,1603090233
21,Kapaa,22.08,-159.32,77.00,83,40,4.70,US,1603090235
41,San Patricio,28.02,-97.52,78.01,88,20,9.17,US,1603090243


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# duplicate df of cities that fulfill conditions to df called hotel_df
hotel_df = ideal_conditions_df

# add Hotel Name column to hotel_df
hotel_df['Hotel Name'] = ['' for i in range(len(hotel_df.index))]

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,Fayaoué,-20.65,166.53,75.13,74,19,9.40,NC,1603090227,
9,Lakkí,37.13,26.85,71.60,78,20,5.82,GR,1603090230,
15,Mahébourg,-20.41,57.70,78.80,65,40,14.99,MU,1603090233,
21,Kapaa,22.08,-159.32,77.00,83,40,4.70,US,1603090235,
41,San Patricio,28.02,-97.52,78.01,88,20,9.17,US,1603090243,


In [35]:
# Set parameters to search for hotels within 5000 meters
for index, row in hotel_df.iterrows():
    params = {
    'location': f"{row['Lat']}, {row['Lng']}",
    'radius': 5000,
    'type': 'hotel',
    'key': g_key
    }
    
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except IndexError:
        hotel_df.loc[index, 'Hotel Name'] = np.nan


In [38]:
# if there's no hotel, drop row 
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != np.nan]

# display 
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,Fayaoué,-20.65,166.53,75.13,74,19,9.40,NC,1603090227,Fayaoue
9,Lakkí,37.13,26.85,71.60,78,20,5.82,GR,1603090230,Agia Marina
15,Mahébourg,-20.41,57.70,78.80,65,40,14.99,MU,1603090233,Mahebourg
21,Kapaa,22.08,-159.32,77.00,83,40,4.70,US,1603090235,Kapaʻa
41,San Patricio,28.02,-97.52,78.01,88,20,9.17,US,1603090243,Sinton


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))